In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [3]:
# trainImage, trainLabels, testImage, testLabels = load_flower_data()

# nb_classes = 2

# # Convert class vectors to binary class matrices.
# trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
# print (trainLabels)
# testLabels = keras.utils.to_categorical(testLabels, nb_classes)
# print (testLabels)
# print (testLabels.shape)

# np.save('../trainImage.npy', trainImage)
# np.save('../trainLabels.npy', trainLabels)
# np.save('../testImage.npy', testImage)
# np.save('../testLabels.npy', testLabels)

# print("[INFO] trainImage matrix: {:.2f}MB".format(
#     (trainImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainLabels matrix: {:.4f}MB".format(
#     (trainLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage matrix: {:.2f}MB".format(
#     (testImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] testLabels matrix: {:.4f}MB".format(
#     (testLabels.nbytes) / (1024 * 1000.0)))


In [4]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
#         self.nb_CNN = [32, 64, 64, 64, 128]  
        self.nb_CNN = [32, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [128] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # train   
        self.global_step = tf.Variable(0, trainable=False)
        self.initial_lr = 0.001 #初始学习率
        self.decay_steps = 10
        self.decay_rate = 0.9
        self.lr = tf.train.exponential_decay( self.initial_lr,
                                     global_step=self.global_step,
                                     decay_steps=self.decay_steps,
                                     decay_rate=self.decay_rate)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300

        # name for checkpoint
        self.model_name = 'CNN_C%d_D%d_Kernel(%d,%d)_%d_lrdecay0.0001' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))    
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y= np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y= np.load('../testLabels.npy')
        
        print ("Training:",self.train_x.shape[0])
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 

            #输入,卷积核为3*3 输出维度为32
            net = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1'               # 命名用于获取变量
                                    )
            print("CNN:",net.get_shape())

            if self.batch_normalization_control:
                net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            net = tf.layers.max_pooling2d(inputs = net,
                                          pool_size = self.pool_size,
                                          strides = (2, 2),
                                          padding='same',
                                          name='pool_conv_' + str(1)
                                         )
            print("CNN:",net.get_shape())
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )

                print("CNN:",net.get_shape())

            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
            
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'_train_hist.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status            
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, 
                                                                                    self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                rand_index = np.random.choice(len(self.train_x), size=batch_size)
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.y: batch_y_tes,
                                                                   self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
            
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
                
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append(test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '_train_hist.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '_train_hist.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
    
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")
        
    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                               self.y: batch_y_tes,
                                                               self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')
                          
        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [5]:
gan_type = 'CNN'
dataset = '4_Flowers'
epoch = 30
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
    #CNN.test(epoch)
        
sess.close()
        

# lrdecay
# Avg per epoch ptime: 3.21, total 100 epochs ptime: 359.54
#  [*] Training finished!
#  [*] Best Epoch:  15 , Accuracy:  0.8199999928474426
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C5_D1_Kernel(3,3)_128_lrdecay/CNN_C5_D1_Kernel(3,3)_128_lrdecay-15
#  [*] Finished testing Best Epoch: 15 , accuracy:  0.8199999928474426 !

# Avg per epoch ptime: 3.30, total 30 epochs ptime: 110.90
#  [*] Training finished!
#  [*] Best Epoch:  15 , Accuracy:  0.8299999833106995
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C4_D1_Kernel(3,3)_128_lrdecay/CNN_C4_D1_Kernel(3,3)_128_lrdecay-15
#  [*] Finished testing Best Epoch: 15 , accuracy:  0.8300000429153442 !

# Avg per epoch ptime: 3.35, total 30 epochs ptime: 112.33
#  [*] Training finished!
#  [*] Best Epoch:  13 , Accuracy:  0.8100000023841858
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_C3_D1_Kernel(3,3)_128_lrdecay0.0001/CNN_C3_D1_Kernel(3,3)_128_lrdecay0.0001-13
#  [*] Finished testing Best Epoch: 13 , accuracy:  0.809999942779541 !

Training: 2000
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 128)
CNN: (100, 16, 16, 128)
CNN: (100, 32768)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN: (100, 128, 128, 32)
CNN: (100, 64, 64, 32)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 128)
CNN: (100, 16, 16, 128)
CNN: (100, 32768)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/beta:0 (float32_ref 32) [32, bytes: 128]
cnn/batch_normalization/gamma:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_2/kernel:0 (float32_ref 3x3x32x64) [18432, bytes: 73728]
cnn/conv_2/bias:0 (float32_ref 64) [64, bytes: 256]
cnn/batch_normalization_1/beta:0 (float32_ref 64) [64, bytes: 256]
cnn/batch_normalization_1/gamma:0

Epoch: [ 6] [  12/  20] time: 21.6415, loss: 0.02236666
Epoch: [ 6] [  13/  20] time: 21.7956, loss: 0.01215334
Epoch: [ 6] [  14/  20] time: 21.9520, loss: 0.06032801
Epoch: [ 6] [  15/  20] time: 22.1075, loss: 0.01378124
Epoch: [ 6] [  16/  20] time: 22.2643, loss: 0.02446437
Epoch: [ 6] [  17/  20] time: 22.4136, loss: 0.03124159
Epoch: [ 6] [  18/  20] time: 22.5734, loss: 0.01595325
Epoch: [ 6] [  19/  20] time: 22.7230, loss: 0.00857198
[6/30] - ptime: 3.2983 loss: 0.01949112 acc: 0.79000 lr: 0.00093874
Epoch: [ 7] [   0/  20] time: 23.5257, loss: 0.01762472
Epoch: [ 7] [   1/  20] time: 23.6735, loss: 0.00724638
Epoch: [ 7] [   2/  20] time: 23.8315, loss: 0.03351281
Epoch: [ 7] [   3/  20] time: 23.9810, loss: 0.01534554
Epoch: [ 7] [   4/  20] time: 24.1374, loss: 0.02744315
Epoch: [ 7] [   5/  20] time: 24.2937, loss: 0.01953941
Epoch: [ 7] [   6/  20] time: 24.4508, loss: 0.03774785
Epoch: [ 7] [   7/  20] time: 24.6107, loss: 0.06087149
Epoch: [ 7] [   8/  20] time: 24.768

Epoch: [13] [  10/  20] time: 47.3688, loss: 0.00154546
Epoch: [13] [  11/  20] time: 47.5196, loss: 0.00344721
Epoch: [13] [  12/  20] time: 47.6781, loss: 0.02722838
Epoch: [13] [  13/  20] time: 47.8270, loss: 0.00013720
Epoch: [13] [  14/  20] time: 47.9865, loss: 0.00678642
Epoch: [13] [  15/  20] time: 48.1396, loss: 0.07440519
Epoch: [13] [  16/  20] time: 48.2956, loss: 0.02350296
Epoch: [13] [  17/  20] time: 48.4560, loss: 0.01452453
Epoch: [13] [  18/  20] time: 48.6176, loss: 0.00059915
Epoch: [13] [  19/  20] time: 48.7732, loss: 0.00353232
[13/30] - ptime: 3.3468 loss: 0.01348067 acc: 0.81000 lr: 0.00087200
Epoch: [14] [   0/  20] time: 49.5541, loss: 0.00275753
Epoch: [14] [   1/  20] time: 49.7159, loss: 0.00824368
Epoch: [14] [   2/  20] time: 49.8770, loss: 0.01212340
Epoch: [14] [   3/  20] time: 50.0270, loss: 0.01338857
Epoch: [14] [   4/  20] time: 50.1830, loss: 0.00509912
Epoch: [14] [   5/  20] time: 50.3333, loss: 0.00222886
Epoch: [14] [   6/  20] time: 50.49

Epoch: [20] [   8/  20] time: 72.9783, loss: 0.00129762
Epoch: [20] [   9/  20] time: 73.1371, loss: 0.01080566
Epoch: [20] [  10/  20] time: 73.2943, loss: 0.02683591
Epoch: [20] [  11/  20] time: 73.4442, loss: 0.00485784
Epoch: [20] [  12/  20] time: 73.6013, loss: 0.00363281
Epoch: [20] [  13/  20] time: 73.7522, loss: 0.00817051
Epoch: [20] [  14/  20] time: 73.9079, loss: 0.00520701
Epoch: [20] [  15/  20] time: 74.0616, loss: 0.00043926
Epoch: [20] [  16/  20] time: 74.2177, loss: 0.00786360
Epoch: [20] [  17/  20] time: 74.3733, loss: 0.00076743
Epoch: [20] [  18/  20] time: 74.5308, loss: 0.00077287
Epoch: [20] [  19/  20] time: 74.6816, loss: 0.00269746
[20/30] - ptime: 3.3105 loss: 0.00713075 acc: 0.78000 lr: 0.00081000
Epoch: [21] [   0/  20] time: 75.4324, loss: 0.00141147
Epoch: [21] [   1/  20] time: 75.5945, loss: 0.00667652
Epoch: [21] [   2/  20] time: 75.7518, loss: 0.00940711
Epoch: [21] [   3/  20] time: 75.9011, loss: 0.00068378
Epoch: [21] [   4/  20] time: 76.05

Epoch: [27] [   6/  20] time: 98.5734, loss: 0.00089693
Epoch: [27] [   7/  20] time: 98.7298, loss: 0.00468132
Epoch: [27] [   8/  20] time: 98.8867, loss: 0.00151335
Epoch: [27] [   9/  20] time: 99.0363, loss: 0.00005929
Epoch: [27] [  10/  20] time: 99.1952, loss: 0.00167537
Epoch: [27] [  11/  20] time: 99.3470, loss: 0.00081660
Epoch: [27] [  12/  20] time: 99.5043, loss: 0.00002220
Epoch: [27] [  13/  20] time: 99.6596, loss: 0.00942311
Epoch: [27] [  14/  20] time: 99.8157, loss: 0.04214340
Epoch: [27] [  15/  20] time: 99.9720, loss: 0.00769900
Epoch: [27] [  16/  20] time: 100.1275, loss: 0.03009879
Epoch: [27] [  17/  20] time: 100.2778, loss: 0.00904062
Epoch: [27] [  18/  20] time: 100.4378, loss: 0.00112682
Epoch: [27] [  19/  20] time: 100.5873, loss: 0.00861149
[27/30] - ptime: 3.3473 loss: 0.00762259 acc: 0.71000 lr: 0.00075241
Epoch: [28] [   0/  20] time: 101.4007, loss: 0.00009169
Epoch: [28] [   1/  20] time: 101.5493, loss: 0.00001556
Epoch: [28] [   2/  20] time: